## General fits for MBI type data

In [ ]:
import sys
import numpy as np

sys.path.append(r'D:/measuring')
sys.path.append(r'D:/measuring/analysis')

from analysis.scripts.mbi import mbi_data_general_fits as fits; reload(fits)
from analysis.lib.m2.ssro import mbi
from analysis.lib.tools import toolbox as tb
reload(mbi); reload(tb)
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import matplotlib.pyplot as plt
reload(fits)
Result = fits.exp_sin(timestamp=  None, contains = '',
                      offset=[0.8], amplitude = [0.1], center = [0], decay_constant = [10], 
                      exp_power = [2],
                      frequency = [0], phase =[0], 
                      fixed = [2,4,5,6],
                      ylim = [0.0,1],
                      plot_fit = True, do_print = True, show_guess = False,
                     correct_ionization = True)

###params
# 0 : offset
# 1 : amplitude
# 2 : center
# 3 : decay_constant
# 4 : exp_power
# 5 : frequency
# 6 : phase

In [ ]:
reload(fits)
Result = fits.exp_sin(timestamp=None, contains = '',
                      offset=[0.5], amplitude = [0.76,0.75], center = [0]*2, decay_constant = [30]*2, exp_power = [2]*2,
                      frequency = [443393.29,88e3], phase =[-179,-185],                       
                      fixed = [0,2,4],
                      ylim = None, plot_fit = True, do_print = True, show_guess = False)

# fit param order for fixing
#0 offset[0], 1 amplitude[0], 2 decay_constant[0], 3 amplitude[0],
#4 frequency[0], 5 phase[0], 6 amplitude[1], 7 frequency[1], 8 phase[1])

# Compare Cr before and after

In [ ]:
reload(fits)
fits.get_CR_histos(contains = '144243')
fits.get_CR_histos(contains = '144058')

In [ ]:
reload(mbi); reload(tb)

folder= tb.latest_data()#older_than='20170720103200')
a=mbi.MBIAnalysis(folder)
a.get_sweep_pts()
a.get_readout_results(name='adwindata')
a.get_electron_ROC()
ax = a.plot_results_vs_sweepparam(ret='ax', name='adwindata')
ax.set_ylim(0.0, 1.0)

minimum_pos = np.argmin(a.p0)
print("minimum at: " + str(a.sweep_pts[minimum_pos]))

closest_to_half_pos = np.argmin(np.abs(a.p0 - 0.5))
print("closest to 0.5 at: %d, value %.3f" % (a.sweep_pts[closest_to_half_pos], a.p0[closest_to_half_pos]))
plt.show()
plt.close('all')

# pos neg data

In [ ]:
import matplotlib.pyplot as plt
reload(mbi); reload(tb)

p_folder= tb.latest_data('positive')
p_a=mbi.MBIAnalysis(p_folder)
p_a.get_sweep_pts()
p_a.get_readout_results(name='adwindata')
p_a.get_electron_ROC()

m_folder= tb.latest_data('negative')
m_a=mbi.MBIAnalysis(m_folder)
m_a.get_sweep_pts()
m_a.get_readout_results(name='adwindata')
m_a.get_electron_ROC()

plt.errorbar(p_a.sweep_pts, p_a.p0, p_a.u_p0, label='pos', fmt='o')
plt.errorbar(m_a.sweep_pts, m_a.p0, m_a.u_p0, label='neg', fmt='o')

diff_x = p_a.sweep_pts
diff_y = p_a.p0 - m_a.p0
diff_y_u = np.sqrt((p_a.u_p0**2) + (m_a.u_p0)**2)

plt.errorbar(diff_x, diff_y , diff_y_u, label='diff', fmt='o')
plt.ylim(-1,1)
plt.legend()
plt.axhline(0)
plt.title(p_a.default_plot_title)

closest_to_zero_pos = np.argmin(np.abs(diff_y))
print("closest to zero at: %d, value %.3f" % (diff_x[closest_to_zero_pos], diff_y[closest_to_zero_pos]))
plt.close('all')

# Pos neg same file data

In [ ]:
import matplotlib.pyplot as plt
reload(mbi); reload(tb)
from analysis.lib.fitting import fit, common
reload(fit)
reload(common)

m_name = "T2star_C7"
tomos = ['X', 'Y']
tomo_data = {}

folder = tb.latest_data(m_name)
a = mbi.MBIAnalysis(folder)

calculate_bloch = True
do_fit = True

for t in tomos:
    a.get_sweep_pts()
    a.get_readout_results(name=t + "_positive")
    a.get_electron_ROC()
    
    x = a.sweep_pts
    pos_y = (a.p0.reshape(-1) - 0.5) * 2
    pos_y_u = a.u_p0.reshape(-1) * 2
    
    a.get_readout_results(name=t + "_negative")
    a.get_electron_ROC()
    
    neg_y = (a.p0.reshape(-1) - 0.5) * 2
    neg_y_u = a.u_p0.reshape(-1) * 2
    
    cor_y = 0.5*pos_y - 0.5*neg_y
    cor_y_u = np.sqrt(pos_y_u**2 + neg_y_u**2) / 2
    
    tomo_data[t] = {
        'x': x,
        'y': cor_y,
        'y_u': cor_y_u
    }
    
if calculate_bloch:
    bloch_length = np.zeros_like(tomo_data[tomos[0]]['y'])
    bloch_length_u = np.zeros_like(tomo_data[tomos[0]]['y_u'])
    for t in tomos:
        bloch_length += tomo_data[t]['y'] ** 2
        bloch_length_u += (tomo_data[t]['y_u'] * tomo_data[t]['y']) ** 2
    bloch_length = np.sqrt(bloch_length) / np.sqrt(0.5*len(tomos))
    bloch_length_u = np.sqrt(bloch_length_u) / bloch_length / np.sqrt(0.5*len(tomos))
    
    tomo_data['bloch'] = {
        'x': tomo_data[tomos[0]]['x'],
        'y': bloch_length,
        'y_u': bloch_length_u
    }
    
    tomos += ['bloch']
    
fit_xvals = np.linspace(0.0e-3, 12.0e-3)

for t in tomos:
    
    
    if do_fit:
        g_f = 300.0
        g_B = None
        fixed = [1]
        fit_method = common.fit_gaussian_decaying_cos
        if t == 'bloch':
            g_f = 0.0
            g_B = 0.0
            fixed=[0,1,3,5]
            fit_method = common.fit_gaussian_decaying_offset_cos
        
        fr = fit.fit1d(
            tomo_data[t]['x'], tomo_data[t]['y'], 
            fit_method,
            g_f, 0.0, 1.0, 0.0, 10.0e-3, g_B, 
            fixed=fixed,
            do_print=True)
        
        ebar = plt.errorbar(
            tomo_data[t]['x'], tomo_data[t]['y'], tomo_data[t]['y_u'],                
            label="%s, T2* = %.2f +/- %.2f ms" % (t, fr['params_dict']['t'] * 1e3, fr['error_dict']['t'] * 1e3), 
            fmt='o'
        )
        plt.plot(fit_xvals, fr['fitfunc'](fit_xvals), color=ebar[0].get_color())
    else:
        ebar = plt.errorbar(tomo_data[t]['x'], tomo_data[t]['y'], tomo_data[t]['y_u'], label=t, fmt='o')

plt.ylim(-1,1)
plt.legend(loc=4)
plt.axhline(0)
plt.title(a.default_plot_title)
plt.show()
plt.close('all')

In [ ]:
f_l = 440e3
mismatch = -10.0
full_rotations = 3

print(360.0*full_rotations*(f_l/(f_l + mismatch) - 1))

print(np.cos(np.pi/2.+2*np.pi*(f_l/(f_l + mismatch) - 1)))